In [6]:
from IPython.core.display import HTML, display
HTML("""
<style>
svg {
    width:40% !important;
    height:40% !important;
}

.container { 
    width:100% !important;
}
</style>
""")

In [7]:
# %autosave 0
%config IPCompleter.greedy=True

In [1]:
import chess

## Evaluieren des Schachbretts & Berechnen attackierter Figuren

Um mögliche Züge der KI bewerten zu können und den besten Zug auszuwählen, müssen diese einen Wert zugewiesen bekommen, der Aussage über die Qualität des Zuges gibt. Dafür können unter anderem die Bewertung des Schachbretts durch Summieren zugewiesener Werte für alle lebenden Figuren oder das Berechnen und Einbeziehen attackierter Figuren zur Hilfe genommen werden.

Dabei ist zunächst jedem Figurentyp ein Wert zuzuweisen. Üblicherweise werden Bauern dabei 1 Punkt, Türmen 5 Punkte, Springern sowie Läufern jeweils 3 Punkte und der Dame 9 Punkte zugeordnet.

In [2]:
PAWN_VALUE = 1
ROOK_VALUE = 5
KNIGHT_VALUE = 3
BISHOP_VALUE = 3
QUEEN_VALUE = 9
KING_VALUE = 20

def assign_piece_value(piece_type):
    return {
        1: PAWN_VALUE,
        2: KNIGHT_VALUE,
        3: BISHOP_VALUE,
        4: ROOK_VALUE,
        5: QUEEN_VALUE,
        6: KING_VALUE
    }[piece_type]

Um den Wert des Schachbretts zu berechnen muss zunächst der Wert aller weißer Figuren berechnet werden und von diesem der Wert aller schwarzen Figuren abgezogen werden. Ist das Ergebnis positiv, verläuft das Spiel günstig für den Spieler der weißen Figuren. Ist das Ergebnis negativ, liegt der Spieler der schwarzen Figuren im Vorteil. Umso höher der Betrag des jeweiligen Spielers ist, desto mehr liegt dieser im Vorteil.

Damit die Werte der Spieler berechnet werden können, wird die Anzahl aller Figurentypen, der jeweiligen Farbe, berechnet und diese mit dem Wert der Figurentypen multipliziert. Am Ende werden die Ergebnisse für alle Figurentypen zusammengezählt und zurückgegeben.

In [3]:
def get_value_by_color(color):
    attacked_pieces_value = map(lambda piece_type : len(board.pieces(piece_type, color)) * assign_piece_value(piece_type), chess.PIECE_TYPES)
    return sum(attacked_pieces_value)

def get_board_value():
    white_value = get_value_by_color(chess.WHITE)
    black_value = get_value_by_color(chess.BLACK)
    
    return white_value - black_value

Die Berechnung der attackierten Figuren ist ähnlich zu dem Vorgehen bei der Berechnung des Brettwerts. Dabei werden erst die Werte, der vom weißen Spieler attackierten Figuren berechnet und davon die Werte der vom schwarzen Spieler attackierten Figuren abgezogen. Auch hierbei ist ein positives Ergebnis zum Vorteil des Spielers der weißen Figuren und ein negatives Ergebnis zum Vorteil des Spielers der schwarzen Figuren. Ebenso gilt umso höher der Wert, desto deutlicher der Vorteil.

Um diese Werte der attackierten Figuren zu berechnen wird jedes Feld durchgegangen. Daraus werden die Felder gefiltert, die von einer Figur der Farbe des Verteidigers belegt sind und von einer Figur der angreifenden Farbe attackiert werden können. Anschließend wird zu diesen Feldern der Typ der Figur zugeordnet, die sich auf dem Feld befindet. Daraufhin werden diesen ihre jeweiligen Werte zugeordnet und diese abschließend summiert.

In [4]:
def get_attacked_pieces_value_by_color(attacker_color, defender_color):
    # filters squares for attacked squares, on which a figure of defender is placed
    attackedSquares = filter(lambda square : board.is_attacked_by(attacker_color, square) and not board.piece_at(square) is None and board.piece_at(square).color is defender_color, chess.SQUARES)
    # maps piece type to attacked figure
    attackedPieces = map(lambda square : board.piece_at(square).piece_type, attackedSquares)
    # maps piece value to attacked pieces
    value = map(assign_piece_value, attackedPieces)
    # sums piece value of all attacked pieces
    return sum(value)
     
def get_attacked_pieces_value():
    white_value = get_attacked_pieces_value_by_color(chess.WHITE, chess.BLACK)
    black_value = get_attacked_pieces_value_by_color(chess.BLACK, chess.WHITE)
    
    return white_value - black_value

Im folgenden Codeausschnitt wird ein Beispiel gegeben wie durch die Verwendung, der zuvor vorgestellten Funktionen, eine Auskunft über den akutellen Spielstand gegeben werden kann.

In [5]:
board = chess.Board()

board_value = get_board_value()
attacked_pieces_value = get_attacked_pieces_value()

# compute the player with a higher value of pieces
if board_value > 0:
    print("White held the lead ahead of Black")
elif board_value < 0:
    print("Black held the lead ahead of White")
else:
    print("Draw")
    
print("Value of the attacked pieces: ", attacked_pieces_value)

Draw
Value of the attacked pieces:  0
